In [4]:
import os
Root = "C:\\Users\\BEHARA AMULYA\\Downloads\\speech emotion"
os.chdir(Root)

In [5]:
ls

 Volume in drive C is Windows
 Volume Serial Number is E8BE-5ED1

 Directory of C:\Users\BEHARA AMULYA\Downloads\speech emotion

15-02-2024  12:29    <DIR>          .
15-02-2024  12:45    <DIR>          ..
25-01-2024  14:20    <DIR>          .ipynb_checkpoints
25-01-2024  14:18         1,338,199 modelForPrediction1.sav
15-02-2024  12:29            21,861 Speech Emotion Recognition.ipynb
25-01-2024  12:41    <DIR>          speech-emotion-recognition-ravdess-data
               2 File(s)      1,360,060 bytes
               4 Dir(s)  24,520,441,856 bytes free


In [6]:
!pip install librosa

In [7]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [8]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))

    return result

In [9]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [10]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(r"C:\Users\BEHARA AMULYA\Downloads\speech emotion\speech-emotion-recognition-ravdess-data\Actor_*\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [11]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [12]:
x_train

array([[-5.22061890e+02,  3.50668907e+01,  3.75342965e+00, ...,
         1.65243153e-04,  1.04321596e-04,  6.55571566e-05],
       [-6.41227722e+02,  4.49487762e+01, -1.85174119e+00, ...,
         3.89261913e-05,  3.05255380e-05,  2.94166657e-05],
       [-6.50705750e+02,  5.30211639e+01, -4.92040443e+00, ...,
         4.75216802e-05,  3.46632514e-05,  1.62844426e-05],
       ...,
       [-5.50096191e+02,  1.70297680e+01, -1.14575634e+01, ...,
         1.51764631e-04,  1.16828531e-04,  8.47479314e-05],
       [-5.55357605e+02,  4.71569710e+01,  1.10750742e+01, ...,
         1.61086457e-04,  1.04962470e-04,  6.52811723e-05],
       [-5.04816345e+02,  3.53618660e+01, -1.43495789e+01, ...,
         6.08151488e-04,  5.55269769e-04,  4.47782222e-04]])

In [13]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [14]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [15]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [16]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [17]:
#Predict for the test set
y_pred=model.predict(x_test)

In [18]:
y_pred

array(['happy', 'calm', 'happy', 'happy', 'disgust', 'calm', 'disgust',
       'happy', 'calm', 'happy', 'happy', 'calm', 'fearful', 'happy',
       'disgust', 'happy', 'calm', 'disgust', 'happy', 'calm', 'calm',
       'disgust', 'disgust', 'calm', 'happy', 'happy', 'calm', 'happy',
       'fearful', 'happy', 'happy', 'fearful', 'happy', 'fearful',
       'happy', 'calm', 'calm', 'fearful', 'calm', 'disgust', 'happy',
       'calm', 'calm', 'calm', 'fearful', 'calm', 'disgust', 'happy',
       'calm', 'happy', 'fearful', 'fearful', 'calm', 'happy', 'happy',
       'happy', 'disgust', 'happy', 'calm', 'calm', 'disgust', 'calm',
       'happy', 'calm', 'happy', 'calm', 'calm', 'disgust', 'fearful',
       'happy', 'fearful', 'fearful', 'fearful', 'fearful', 'fearful',
       'disgust', 'fearful', 'happy', 'calm', 'fearful', 'disgust',
       'calm', 'fearful', 'calm', 'disgust', 'happy', 'disgust', 'happy',
       'happy', 'fearful', 'disgust', 'calm', 'calm', 'happy', 'disgust',
      

In [19]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 71.88%


In [20]:
from sklearn.metrics import accuracy_score, f1_score

In [21]:
f1_score(y_test, y_pred,average=None)

array([0.86206897, 0.60759494, 0.67567568, 0.67826087])

In [22]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,calm,calm
2,happy,happy
3,happy,happy
4,disgust,disgust
5,calm,calm
6,happy,disgust
7,happy,happy
8,disgust,calm
9,happy,happy


In [23]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [24]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature(r"C:\Users\BEHARA AMULYA\Downloads\speech emotion\speech-emotion-recognition-ravdess-data/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['happy'], dtype='<U7')